In [9]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

glueContext = GlueContext(SparkContext.getOrCreate())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
salesDF = glueContext.create_dynamic_frame.from_catalog(
             database="amitdbetl",
             table_name="sales")
customerDF = glueContext.create_dynamic_frame.from_catalog(
             database="amitdbetl",
             table_name="customers")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
customerDF.toDF().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------------+--------------------+-------------+---------+---------+----------------+---------------+
|customerid|        customername|               email|         city|  country|territory|contactfirstname|contactlastname|
+----------+--------------------+--------------------+-------------+---------+---------+----------------+---------------+
|         1|   Land of Toys Inc.|  gladys.rim@rim.org|          NYC|      USA|       NA|           James|           Butt|
|         2|  Reims Collectables|yuki_whobrey@aol.com|        Reims|   France|     EMEA|       Josephine|        Darakjy|
|         3|     Lyon Souveniers|fletcher.flosi@ya...|        Paris|   France|     EMEA|             Art|         Venere|
|         4|   Toys4GrownUps.com| bette_nicka@cox.net|     Pasadena|      USA|       NA|           Lenna|       Paprocki|
|         5|Corporate Gift Id...|     vinouye@aol.com|San Francisco|      USA|       NA|         Donette|         Foller|
|         6|Technics Sto

---Update the Data in the Dynamicframe from the customers table in the amitdbetl database.

In [18]:
customersDF = glueContext.create_dynamic_frame.from_catalog(
             database="amitdbetl",
             table_name="customers")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
def addcolumns(rec):
    rec["contactfullname"] = {}
    rec["contactfullname"] = rec["contactfirstname"] + ' ' + rec["contactlastname"]
    return rec

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
customersDF =  Map.apply(customersDF, f = addcolumns)
customersDF.toDF().show();

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+-------------+---------------+-----------------+---------+--------------------+----------------+----------+
|territory|        customername|         city|contactlastname|  contactfullname|  country|               email|contactfirstname|customerid|
+---------+--------------------+-------------+---------------+-----------------+---------+--------------------+----------------+----------+
|       NA|   Land of Toys Inc.|          NYC|           Butt|       James Butt|      USA|  gladys.rim@rim.org|           James|         1|
|     EMEA|  Reims Collectables|        Reims|        Darakjy|Josephine Darakjy|   France|yuki_whobrey@aol.com|       Josephine|         2|
|     EMEA|     Lyon Souveniers|        Paris|         Venere|       Art Venere|   France|fletcher.flosi@ya...|             Art|         3|
|       NA|   Toys4GrownUps.com|     Pasadena|       Paprocki|   Lenna Paprocki|      USA| bette_nicka@cox.net|           Lenna|         4|
|       NA|Corporate

In [22]:
customersDF = DropFields.apply(customersDF,["contactfirstname","contactlastname"])
customersDF.toDF().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+-------------+-----------------+---------+--------------------+----------+
|territory|        customername|         city|  contactfullname|  country|               email|customerid|
+---------+--------------------+-------------+-----------------+---------+--------------------+----------+
|       NA|   Land of Toys Inc.|          NYC|       James Butt|      USA|  gladys.rim@rim.org|         1|
|     EMEA|  Reims Collectables|        Reims|Josephine Darakjy|   France|yuki_whobrey@aol.com|         2|
|     EMEA|     Lyon Souveniers|        Paris|       Art Venere|   France|fletcher.flosi@ya...|         3|
|       NA|   Toys4GrownUps.com|     Pasadena|   Lenna Paprocki|      USA| bette_nicka@cox.net|         4|
|       NA|Corporate Gift Id...|San Francisco|   Donette Foller|      USA|     vinouye@aol.com|         5|
|       NA|Technics Stores Inc.|   Burlingame|   Simona Morasca|      USA|  gladys.rim@rim.org|         6|
|     EMEA|Daedalus Designs ...|     

In [23]:
def parsedomain(rec):
    rec["domain"] = {}
    indx = rec["email"].index('@')
    rec["domain"] = rec["email"][indx + 1:]
    return rec

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
customersDF =  Map.apply(customersDF, f = parsedomain)
customersDF.toDF().show();

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+---------+-------------+-----------------+---------+--------------------+----------+
|territory|        customername|   domain|         city|  contactfullname|  country|               email|customerid|
+---------+--------------------+---------+-------------+-----------------+---------+--------------------+----------+
|       NA|   Land of Toys Inc.|  rim.org|          NYC|       James Butt|      USA|  gladys.rim@rim.org|         1|
|     EMEA|  Reims Collectables|  aol.com|        Reims|Josephine Darakjy|   France|yuki_whobrey@aol.com|         2|
|     EMEA|     Lyon Souveniers|yahoo.com|        Paris|       Art Venere|   France|fletcher.flosi@ya...|         3|
|       NA|   Toys4GrownUps.com|  cox.net|     Pasadena|   Lenna Paprocki|      USA| bette_nicka@cox.net|         4|
|       NA|Corporate Gift Id...|  aol.com|San Francisco|   Donette Foller|      USA|     vinouye@aol.com|         5|
|       NA|Technics Stores Inc.|  rim.org|   Burlingame|   Simon

In [25]:
customersDF = customersDF.rename_field("territory","region")
customersDF.toDF().show();

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+---------+--------------------+---------+-----------------+------+--------------------+----------+
|         city|   domain|        customername|  country|  contactfullname|region|               email|customerid|
+-------------+---------+--------------------+---------+-----------------+------+--------------------+----------+
|          NYC|  rim.org|   Land of Toys Inc.|      USA|       James Butt|    NA|  gladys.rim@rim.org|         1|
|        Reims|  aol.com|  Reims Collectables|   France|Josephine Darakjy|  EMEA|yuki_whobrey@aol.com|         2|
|        Paris|yahoo.com|     Lyon Souveniers|   France|       Art Venere|  EMEA|fletcher.flosi@ya...|         3|
|     Pasadena|  cox.net|   Toys4GrownUps.com|      USA|   Lenna Paprocki|    NA| bette_nicka@cox.net|         4|
|San Francisco|  aol.com|Corporate Gift Id...|      USA|   Donette Foller|    NA|     vinouye@aol.com|         5|
|   Burlingame|  rim.org|Technics Stores Inc.|      USA|   Simona Morasca|    NA|  glady